# Generate Training Frames with Pretrained Model
This is a workflow to use a pretrained YOLO shark tracker to exrtact shark frames from a youtube video, double check them and use them as training frames for a more advanced model.

1. Run yolo with source=youtube video
2. Generate 1fps annotations from yolo results
3. Extract 1fps frames from video
4. Upload video and annotations on VIAME
5. Double check them [local]
6. Download new annotations [local]

In [1]:
from ultralytics import YOLO

## 1. Run yolo with source=youtube video

In [19]:
video = 'https://youtu.be/kIG__GO175o'
model_path = '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt'

CONF_TRESHOLD = 0.2

In [21]:
model = YOLO(model_path)
results = model.track(source=video, persist=True, conf=CONF_TRESHOLD)


1/1: https://youtu.be/kIG__GO175o... Success ✅ (2877 frames of shape 1920x1080 at 29.97 FPS)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 384x640 (no detections), 14.0ms
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
0: 384x640 1 shark, 9.1ms
0: 384x640 1 shark, 6.7ms
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting for stream 0
WARNING ⚠️ Waiting 

KeyboardInterrupt: 